**subsetMisuse.ipynb**

Misuse of subset annotation property.

**Documentation:** https://github.com/edamontology/edamverify/blob/master/docs/subsetMisuse.md

Note: ...    

Set constants for script return values. Load EDAM_dev.owl from GitHub into an RDF graph.

In [ ]:
import io
import sys
from rdflib import ConjunctiveGraph, Namespace

# Constants for script return value as per https://github.com/edamontology/edamverify.
NOERR = 0
INFO  = 1
WARN  = 2
ERROR = 3

#Load EDAM_dev.owl from GitHub into an RDF graph.
print("Loading graph ...", end="")
g = ConjunctiveGraph()
# g.load('https://raw.githubusercontent.com/edamontology/edamontology/master/EDAM_dev.owl', format='xml')
g.load('EDAM_dev.owl')
g.bind('edam', Namespace('http://edamontology.org#'))
print("done!")

Define SPARQL query to extract ID, term and subset of all concepts. Run the query.

In [ ]:
# Compile SPARQL query
query_term = """
SELECT ?id ?term ?subset ?deprecated WHERE
{
?id rdfs:label ?term .
?id oboInOwl:inSubset ?subset . 
OPTIONAL {?id owl:deprecated ?deprecated .}
}
"""

# Declare hash tables for results
ids = {}
numerical_ids = {}
terms = {}
subsets = {}

# Run SPARQL query and collate results
errfound = False    
report = list()
results = g.query(query_term)

Analyse results of query.

**NOTE**
1. Concepts which are not defined to be in one of the "topics", "operations", "data" or "formats" subsets are ignored (not checked).

In [ ]:
for r in results :
#    print(str(r['id']), str(r['term']), str(r['subset']))
    
    id     = str(r['id'])
    term   = str(r['term']) 
    subset = str(r['subset']) 
    deprecated = str(r['deprecated'])

    # Discard concepts in irrelevant subsets
    if subset != "http://purl.obolibrary.org/obo/edam#topics" \
            and subset != "http://purl.obolibrary.org/obo/edam#operations" \
            and subset != "http://purl.obolibrary.org/obo/edam#data" \
            and subset != "http://purl.obolibrary.org/obo/edam#formats":
        continue
        
    
    # Check value of <deprecated> is "true" only 

    # Check for mismatch between subset and subontology (non-deprecated concepts)    

    # Check subset is correct for deprecated concepts 

 
 